In [1]:
import sys
sys.path.append("./../")
from evaluation.gsm8k import judge
import json
from thirdparty.openai.grade_school_math.dataset import read_jsonl
import numpy as np
from scipy.stats import bootstrap
import os

In [2]:
reference = read_jsonl("../data/gsm8k_jsonl/gsm8k/test.json")

In [3]:
def clean(text):
    texts = text.split("####")
    if len(texts) == 1:
        return text
    last_txt = "#### " + "".join(texts[-1].split(" "))
    return "####".join(texts[:-1]) + last_txt


names = os.listdir("./../result")
name2data = {}
for name in names:
    with open(f"./../result/{name}", "r") as f:
        data = json.load(f)
        
    for entry in data:
        if "clean_output" in entry:
            entry["answer"] = clean(entry["clean_output"])
        else:
            entry["answer"] = entry["output"]
    name2data[name] = data

In [4]:
reses = []
for name, data in name2data.items():
    
    if len(data) != len(reference) or name == "mistral_v1_outputs_train.json":
        continue
    results = judge(data, reference)
    
    results = np.array(results).reshape((1,-1))
    
    ci = bootstrap(results, np.mean).confidence_interval
        
    reses.append((f"{name}: {np.mean(results)}, {ci}, {len(data)} samples", results))

reses.sort(key= lambda x: x[1].mean(),reverse = True)
for res,val in reses:
    print(res)

pretrain_only_mistral.json: 0.4200151630022744, ConfidenceInterval(low=0.39423805913570886, high=0.44655041698256254), 1319 samples
pretrain_mistral_outer_loop_3.json: 0.4071266110689917, ConfidenceInterval(low=0.38059135708870356, high=0.43442001516300227), 1319 samples
pretrain_mistral_outer_loop_0.json: 0.3957543593631539, ConfidenceInterval(low=0.3692191053828658, high=0.42153146322971946), 1319 samples
pretrain_mistral_outer_loop_2.json: 0.38514025777103866, ConfidenceInterval(low=0.3593631539044731, high=0.4116755117513268), 1319 samples
mistral_v1_outputs_test.json: 0.3813495072024261, ConfidenceInterval(low=0.354814253222138, high=0.40788476118271416), 1319 samples
mistral_v1_outputs_test_pause_token.json: 0.3639120545868082, ConfidenceInterval(low=0.3373768006065201, high=0.38968915845337376), 1319 samples
pretrain_mistral_no_gt_outer_loop_2.json: 0.35860500379075055, ConfidenceInterval(low=0.332827899924185, high=0.38514025777103866), 1319 samples
ll_reward_3_samps_rollout_no

In [14]:
name = "pretrain_only_mistral.json"
name2 = "wsft_pause_constrained_ll_reward_2_samps_rollout_0.json"
index = 7

data = name2data[name]
sample = name2data[name][index]
sample2 = name2data[name2][index]
ref_sample = reference[index]
print("#"*100)
print("question:")
print(ref_sample["question"])
print("#"*100)
print("Ground Truth")
print(ref_sample["answer"])
print("#"*100)
print("Prediction round 1")
print(sample["output"])
print("#"*100)
print("Clean round 1")
print(sample["clean_output"])
print("#"*100)
print(judge(data, reference)[index])
# print("Prediction round 0")
# print(sample2["output"])

####################################################################################################
question:
Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?
####################################################################################################
Ground Truth
First find how many gigabytes are in 40% of the file: 200 GB * 40% = <<200*40*.01=80>>80 GB
Then divide that number by the download rate to find the time until Windows restarts: 80 GB / 2 GB/minute = <<80/2=40>>40 minutes
Then find the time to download the whole file after the restart: 200 GB / 2 GB/minute = <<200/2=100>>100 minutes
Then add the time to download 40% of the file, to download the whole file, and to wait for Windows to update: 40 minutes + 100 minutes + 20 minutes = <<

In [108]:
# ll_name2data = list(
#     filter(
#         lambda x: "ll_reward_3_samps" in x,
#         name2data
#     )
# )
# ll_name2data = {k: name2data[k] for k in ll_name2data}
ll_name2data = name2data
name1 = "ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_5.json"
name2 = "mistral_v1_outputs_test_pause_token.json"
judge1 = judge(ll_name2data[name1], reference)
judge2 = judge(ll_name2data[name2], reference)
j1_c = 0
j2_c = 0
for index,(j1,j2) in enumerate(zip(judge1,judge2)):
    if j1 and not j2:
        j1_c +=1
    if j2 and not j1:
        j2_c += 1
    # if j1 and not j2:
    #     print("#"*100)
    #     print(name1, "correct")
    #     print("-"*100)
    #     print("-"*100)
    #     print(ll_name2data[name1][index]["output"])
    #     print("-"*100)
    #     print("+"*100)
    #     print(name2, "inccorrect")
    #     print("-"*100)
    #     print("-"*100)
    #     print(ll_name2data[name2][index]["output"])
    #     print("-"*100)
    #     print("#"*100)
print(name1, "samples exculisively correct", j1_c)
print(name2, "samples exculisively correct", j2_c)

ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_5.json samples exculisively correct 177
mistral_v1_outputs_test_pause_token.json samples exculisively correct 193


In [42]:
def sf(text):
    texts = text.split("####")
    if len(texts) == 1:
        return text
    last_txt = "#### " + "".join(texts[-1].split(" "))
    return "####".join(texts[:-1]) + last_txt
for dp in data:
    dp["answer"] = sf(dp["answer"])
    

In [47]:
data[index]["answer"]

' He produces 252 eggs x 7 days = <<252*7 =176 4 >>1764 eggs\nThat means he sells 1764 eggs / 12 = << 1764/12=147>>147 dozen\nHe then makes 14 7 dozen x $2/dozen = $<<1 47*2=294 >> 294 in the week\n#### 294'